In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from nltk.tokenize import RegexpTokenizer
import re
from tqdm import tqdm
import re 
from gensim import corpora
import gensim 
import pyLDAvis.gensim_models 
from collections import Counter 
from job import filter_data

In [45]:
stopword_list = pd.read_excel('Word/stopword_list.xlsx')
one_char_keyword = pd.read_excel('Word/one_char_list.xlsx')

In [46]:
def remove_stopword(tokens):
    review_removed_stopword = []
    for token in tokens:
        # 토큰의 글자 수가 2글자 이상인 경우
        if 1 < len(token):
            # 토큰이 불용어가 아닌 경우만 분석용 리뷰 데이터로 포함
            if token not in list(stopword_list['stopword']):
                review_removed_stopword.append(token)
        # 토큰의 글자 수가 1글자인 경우
        else:
            # 1글자 키워드에 포함되는 경우만 분석용 리뷰 데이터로 포함
            if token in list(one_char_keyword['one_char_keyword']):
                review_removed_stopword.append(token)
    return review_removed_stopword

In [62]:
def lda_modeling(review_prep, NUM_TOPICS=5, PASSES=15):
    dictionary = corpora.Dictionary(review_prep)
    corpus = [dictionary.doc2bow(review) for review in review_prep]
    model = gensim.models.ldamodel.LdaModel(corpus, 
                                            num_topics = NUM_TOPICS, 
                                            id2word = dictionary, 
                                            passes = PASSES)
    return model, corpus, dictionary

In [63]:
def lda_visualize(model, corpus, dictionary, month, job):
    pyLDAvis.enable_notebook()
    result_visualized = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    pyLDAvis.display(result_visualized)
    RESULT_FILE = f'result/lda_result_{month}월_{job}.html'
    pyLDAvis.save_html(result_visualized, RESULT_FILE)

In [64]:
df = pd.read_csv('data/jobs.csv')
filtered_data, months = filter_data(df)

In [71]:
for i in range(len(months)):
    filtered_month_data = filtered_data[filtered_data['date'].dt.month==months[i]]
    grouped_data = filtered_month_data.groupby('job').agg({
            'title': 'count',
            'reply': 'sum',
            'view': 'sum',
            'recommend': 'sum'
        }).reset_index()
    sum_data = grouped_data[['title','reply','view','recommend']].sum(axis=1)
    top_jobs_index = sum_data.nlargest(3).index
    bottom_jobs_index = sum_data.nsmallest(3).index
    filtered_month_data['remove_reply'] = filtered_month_data['title'].apply(lambda x: re.sub('[^가-힣 ]', '', x))
    tokenizer = RegexpTokenizer("[가-힣]+") 
    filtered_month_data['tokens']=filtered_month_data['remove_reply'].apply(tokenizer.tokenize)
    filtered_month_data['tokens'] = filtered_month_data['tokens'].apply(remove_stopword)

    top_job_list = grouped_data['job'][top_jobs_index].values
    bottom_job_list = grouped_data['job'][bottom_jobs_index].values

    for j in range(len(top_job_list)):
        top_job_tokens= filtered_month_data[filtered_month_data['job']==top_job_list[j]]['tokens']
        bottom_job_tokens= filtered_month_data[filtered_month_data['job']==bottom_job_list[j]]['tokens']

        model, corpus, dictionary = lda_modeling(top_job_tokens)
        lda_visualize(model, corpus, dictionary, months[i], top_job_list[j])
        model, corpus, dictionary = lda_modeling(bottom_job_tokens)
        lda_visualize(model, corpus, dictionary, months[i], bottom_job_list[j])

C:\Users\SS\AppData\Local\Temp\ipykernel_26060\450259476.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_month_data['remove_reply'] = filtered_month_data['title'].apply(lambda x: re.sub('[^가-힣 ]', '', x))
C:\Users\SS\AppData\Local\Temp\ipykernel_26060\450259476.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_month_data['tokens']=filtered_month_data['remove_reply'].apply(tokenizer.tokenize)
C:\Users\SS\AppData\Local\Temp\ipykernel_26060\450259476.py:15: SettingWithCopyWarning: 
A 